# Tensorboard Hyperparameter Tunning

Experiment setup and the HParams experiment summary

Experiment with  hyperparameters in the model:

1. Number of units in the first dense layer
2. Dropout rate in the dropout layer
3. Optimizer

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn import datasets, metrics, model_selection


In [ ]:
# clear previous log
# rm -rvf logs

# Clear any logs from previous runs
# rm -rf 'logs/ANN/fit/'

# rm -rvf logdir logs/ANN/fit/

# Clear any logs from previous runs
rm -rf ./logs/

In [3]:
# read data

xlsx = pd.ExcelFile('./Data/Outlier_thresh1_50.xlsx')
df = pd.read_excel(xlsx, '1S 1X')
dataset= df.copy()

# Split the data into train and test

# the 'sample' method takes a certain fraction randomly
#Note that we use `random_state` to ensure the reproducibility of the examples.
train_dataset = dataset.sample(frac=0.7, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


# split datat into input and target

train_input = train_dataset.copy()
test_input = test_dataset.copy()

train_target = train_input.pop('RHOB')
test_target = test_input.pop('RHOB')



In [ ]:
# Load the TensorBoard notebook extension
# %reload_ext tensorboard

%load_ext tensorboard


In [ ]:
# logdir = "logs\ANN"


from tensorboard.plugins.hparams import api as hp

Experiment setup and the HParams experiment summary

Experiment with  hyperparameters in the model:

1. Number of units in the first dense layer
2. Dropout rate in the dropout layer
3. Optimizer

In [ ]:
# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([300, 200,512]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 16, 32,64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

In [ ]:
# help(hp.HParam)

# dir(hp)

# help(    tf.keras.layers.Flatten())
# help(hparams)

In [ ]:
METRIC = "mse"

with tf.summary.create_file_writer('logs/ANN/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC, display_name='mse')],
  )

In [ ]:
# Adapt TensorFlow runs to log hyperparameters and metrics

def create_model(hparams):
    
#normalization
    normalizer=preprocessing.Normalization()
    normalizer.adapt(np.array(test_input))
    
    model = tf.keras.models.Sequential([
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu', input_shape=(5,)),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(1),
  ])
    
    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='mean_absolute_error',
      metrics=['mse'],
  )
    
#     logdir= 'logs/ANN/hparam_tuning/'
#     model.fit(train_input, train_target,
#               epochs=10,
#                callbacks=[
#         tf.keras.callbacks.TensorBoard(logdir),  # log metrics
#         hp.KerasCallback(logdir, hparams),  # log hparams
#     ],
#               ) # Run with a few epoch to speed things up for demo purposes
    
    loss, mse = model.evaluate(test_input, test_target)
    return mse





In [ ]:
# For each run, log an hparams summary with the hyperparameters and final mse:

def experiment(experiment_dir, hparams):
     with tf.summary.create_file_writer(experiment_dir).as_default():
        hp.hparams(hparams)
        mse = create_model(hparams)
        tf.summary.scalar(METRIC, mse, step=1)


In [ ]:
# or use keras callbacks when traing

logdir= 'logs/ANN/hparam_tuning/'

model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)

In [ ]:
experiment_no = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,}

            experiment_name = f'Experiment {experiment_no}'
            print(f'Starting Experiment: {experiment_name}')
            print({h.name: hparams[h] for h in hparams})
            experiment('logs/ANN/hparam_tuning/' + experiment_name, hparams)
            experiment_no += 1

4. Visualize the results in TensorBoard's HParams plugin

In [ ]:
%tensorboard -- logdir= "logs/ANN/hparam_tuning/" --port=6005

# using sckit-learn for Tuning

Tunables in ANN
1. Batch and Epoch
2. Optimization algorithm
3. Learning Rate and Momentum
4. Weight initialization
5. Activation Functions
6. Neuron Hidden layer

In [4]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasRegressor

In [5]:
import keras
# dir(keras.wrappers.scikit_learn)

### part 1 - Tuning batch and epoch

In [7]:
# read data

xlsx = pd.ExcelFile('./Data/Outlier_thresh1_50.xlsx')
df = pd.read_excel(xlsx, '1S 5X')
dataset= df.copy()


# Split the data into train and test

# the 'sample' method takes a certain fraction randomly
#Note that we use `random_state` to ensure the reproducibility of the examples.
train_dataset = dataset.sample(frac=0.7, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


# split datat into input and target

train_input = train_dataset.copy()
test_input = test_dataset.copy()

train_target = train_input.pop('RHOB')
test_target = test_input.pop('RHOB')


KeyError: 'RHOB'

In [ ]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(32, input_dim=5, activation='relu'))
    mymodel.add(Dense(1))
    
    # Compile the model
    mymodel.compile(loss='mae', optimizer='adam', metrics=['mse'])
    return mymodel

In [ ]:
# create model
model = KerasRegressor(build_fn=create_my_model)

In [ ]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [5,10, 30, 50,70,100,150]

In [ ]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(train_input, train_target)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## part 2- Optimizing Activation Function

In [ ]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(32, input_dim=5, activation='relu'))
    mymodel.add(Dense(1))
    # Compile model
    mymodel.compile(loss='mae', optimizer=optimizer, metrics=['mse'])
    return mymodel

In [ ]:
# create model
model = KerasRegressor(build_fn=create_my_model, epochs=50, batch_size=40)

In [ ]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam','Adamax']
parameter_grid = dict(optimizer=optimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_input, train_target)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
Best Params={'batch_size': 40, 'epochs': 50, 'optimizer': 'SGD'}

# Using Keras-Tuner
keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

#### Hyperparameters
1. How many number of hidden layers we should have? <br>
    <em>Dense vrs dropout</em>
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
# clear previous log
# rm -rvf Test_weights

# Clear any logs from previous runs
# rm -rf 'logs/ANN/fit/'

# rm -rvf logdir Test_weights

# Clear any logs from previous runs
# rm -rf Test_weights

In [2]:
def build_model(hp):
    model = keras.Sequential()

    
    for i in range(hp.Int('num_layers', 1, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=2,
                                            max_value=512,
                                            step=8,            
#                                             sampling="log"
                                           ),
                                            activation= 'relu',
                                            
#                                activation=hp.Choice('act_' + str(i), ['relu'])#, 'ELU', 'maxout', 'Leaky Relu' # adding other methods worsens the results
                               
#                                 kernel_regularizer=hp.Choice('act_' + str(i), ['l1', 'l2']) #is not helping at all
                               
                              ))
        
        model.add(layers.Dropout(rate=hp.Float('D_rate_'+ str(i),
                                              min_value=0.0,
                                              max_value=1,
                                              step=0.1
#                                               sampling="log"
                                              ),       
                                ))

#         model.add(layers.Dropout(rate=hp.Choice('dropout rate_'+ str(i),
#                                              [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
#                                 ))
    model.add(layers.Dense(1,
#                            activation='linear'
                          ))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [0.15, 0.1, 1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mse'])
    return model

In [4]:
tuner = RandomSearch(
    build_model,
    objective =  'val_loss',   #   'val_mean_absolute_error', #(['loss', 'mse', 'val_loss', 'val_mse']
    max_trials=10,
    executions_per_trial=6,
    directory='Test_weights',
    project_name='ANNTuning')

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 512, 'step': 8, 'sampling': None}
D_rate_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.15, 'conditions': [], 'values': [0.15, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}


In [8]:
from tensorflow.keras.layers.experimental import preprocessing
#     #normalization
# normalizer=preprocessing.Normalization()
#     #then adapt it to the data
# norm_train_input= normalizer.adapt(np.array(train_input))
    
tuner.search(train_input, train_target,
             epochs=50,
             validation_data=(test_input, test_target))

Trial 5 Complete [00h 05m 14s]
val_loss: 0.15094660222530365

Best val_loss So Far: 0.15033872425556183
Total elapsed time: 00h 15m 06s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
num_layers        |13                |4                 
units_0           |394               |322               
D_rate_0          |0.2               |0.7               
learning_rate     |0.001             |0.001             
units_1           |282               |26                
D_rate_1          |0.9               |0.6               
units_2           |322               |458               
D_rate_2          |0.9               |0.6               
units_3           |306               |18                
D_rate_3          |0.4               |0.2               
units_4           |354               |138               
D_rate_4          |0.9               |0.7               
units_5           |194               |178               
D_rate_5          |0.2               |0

ValueError: in user code:

    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    D:\anaconda3\lib\site-packag

In [ ]:
tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=2)
models

# Test run

Trial summary
Hyperparameters:
num_layers: 1
units_0: 34
learning_rate: 0.001
units_1: 482
units_2: 418
units_3: 258
units_4: 450
units_5: 290
units_6: 322
units_7: 354
units_8: 130
units_9: 482
units_10: 354
units_11: 66
units_12: 386
units_13: 322
units_14: 66
Score: 0.12417174875736237

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# A function to hold preprocessor, layers, model and compiler 
def FFBackProp(inputs, output):
   

#     #normalization
#     normalizer=preprocessing.Normalization()
#     #then adapt it to the data
#     normalizer.adapt(np.array(inputs))
    
    
    model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation="relu", input_shape=(5,), 
#                           kernel_regularizer="l2"
                         ),
    tf.keras.layers.Dropout(0.2),
        
    tf.keras.layers.Dense(64,activation="relu", 
#                           kernel_regularizer="l2"
                         ),
    tf.keras.layers.Dropout(0.1),
        
    tf.keras.layers.Dense(32,activation="relu", 
#                           kernel_regularizer="l2"
                         ),
#     tf.keras.layers.Dense(32,activation="relu", 
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.2),
        
#     tf.keras.layers.Dense(64,activation="relu",
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(64,activation="relu",
# #                           kernel_regularizer="l2"
#                          ),
#     tf.keras.layers.Dropout(0.2),
        
    
        #output layer
    tf.keras.layers.Dense(1),
  ]
    )
    
    
#     compile the model
    model1.compile(optimizer='adam',
                loss='mean_absolute_error', #(from_logits=True),
                metrics=["mse"]
               )
    
    model1.summary()
    
    return model1
    
myFFBP= FFBackProp(train_input,train_target)


In [ ]:

myFFBPtraining = myFFBP.fit(
                            train_input, train_target,
                            # Calculate validation results on 30% of the training data
                            validation_split=0.3,


                            validation_data=(test_input, test_target),


                            # suppress logging
                            verbose=0,

                            epochs=50,

                            batch_size= 40


                            #early stopping
                            #the fuction to stop the trianing by tracking the validation loss
                            #callbacks= keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

                            #to store callbacks in logs
                        #     callbacks =[tb_callback]
)


myFFBP.evaluate(test_input, test_target,
#                 callbacks=[tb_callback],
                verbose=0, )

## plot results

In [ ]:
from matplotlib import pyplot as plt

def plot_loss(arg):
  plt.plot(arg.history['loss'], label='loss')
  plt.plot(arg.history['val_loss'], label='val_loss')
#   plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error [RHOB]')
  plt.legend()
  plt.grid(True)


plot_loss(myFFBPtraining)

In [ ]:
predicted = myFFBP.predict(test_input)

fig, ax = plt.subplots()
ax.scatter(test_target, predicted, alpha=0.5)

ax.set_xlabel('True Values')
ax.set_ylabel('Predicted')


In [ ]:
# using sklearn
from sklearn import metrics
print("r2 score: {}".format(metrics.r2_score(test_target_actual, test_target_predicted)))
print("mse: {}".format(metrics.mean_squared_error(test_target_actual, test_target_predicted)))
print("rmse: {}".format(np.sqrt(metrics.mean_squared_error(test_target_actual, test_target_predicted))))
print("mae: {}".format(metrics.mean_absolute_error(test_target_actual, test_target_predicted)))

## predict from a diff well

In [ ]:
xlsx = pd.ExcelFile('./Data/Outliers_out.xlsx')
df2 = pd.read_excel(xlsx, '1x 4s')
dataset2= df2.copy()


# split datat into input and target

inputs2 = dataset2.copy()
target2 = inputs2.pop('RHOB')

In [ ]:
# predicted = model.predict(test_input)

predicted2 = myFFBP.predict(inputs2)
fig, ax= plt.subplots()
ax.scatter(target2,predicted2, alpha=0.5)
ax.set_xlabel("True")
ax.set_ylabel("predicted")

In [ ]:
print("r2 score: {}".format(metrics.r2_score( target2, predicted2)))
print("mse: {}".format(metrics.mean_squared_error( target2, predicted2)))
print("rmse: {}".format(np.sqrt(metrics.mean_squared_error( target2, predicted2))))
print("mae: {}".format(metrics.mean_absolute_error( target2, predicted2)))